# Notebook 3: The Ethologist's Toolkit - Advanced Feature Engineering

**Overall Goal:** To dramatically improve our model's performance by creating a rich set of engineered features. Instead of feeding the model raw coordinates, we will provide it with meaningful, pre-calculated information about the mice's speed, orientation, and spatial relationships.

**The Strategy:**
We will use the same LightGBM model and frame-by-frame approach as in [Notebook 2](https://www.kaggle.com/code/wafaaalayoubi/2-baseline-lightgbm-0-96). The *only* thing we will change is the input data. By comparing the performance of this new model to our baseline, we can directly measure the impact of our feature engineering.

*   **Feature Categories:**
    1.  **Kinematic Features:** Describe the movement of each mouse individually (e.g., speed of different bodyparts).
    2.  **Interaction Features:** Describe the relationship *between* mice (e.g., distance between noses, relative speed).
    3.  **Postural Features:** Describe the shape or posture of each mouse (e.g., how stretched out its body is).
*   **Model:** We will use the same `LGBMClassifier`.
*   **Evaluation:** We will compare the `macro avg f1-score` of this model directly against the **0.49** we achieved in [Notebook 2](https://www.kaggle.com/code/wafaaalayoubi/2-baseline-lightgbm-0-96).

# Step 1: Setup and Feature Engineering Functions

**Goal:** To set up our environment and create a robust function for feature engineering. We will start with the same `load_and_process_video` function from the previous notebook and then build a new, powerful `create_advanced_features` function on top of it.

**Action:**
1.  Import libraries and reuse our `load_and_process_video` function.
2.  Define a list of core anatomical bodyparts to focus on, ignoring experiment-specific ones like 'headpiece'.
3.  Create the `create_advanced_features` function that will contain all our feature creation logic. We will build this function up with kinematic, interaction, and postural features.

In [ ]:
# --- Core Libraries ---
import pandas as pd
import numpy as np
import os
import seaborn as sns
from tqdm.auto import tqdm
import warnings

# --- Modeling Libraries ---
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [ ]:
# --- Set display options ---
pd.set_option('display.max_columns', 200)
sns.set_style('whitegrid')

# --- Define Constants and Paths ---
DATA_PATH = '/kaggle/input/MABe-mouse-behavior-detection' 

warnings.filterwarnings("ignore")

In [ ]:
# --- Reusable Data Loading Function (from Notebook 2) ---
def load_and_process_video(video_id, lab_id, data_path):
    tracking_path = os.path.join(data_path, 'train_tracking', lab_id, f'{video_id}.parquet')
    if not os.path.exists(tracking_path):
        return None
    df_long = pd.read_parquet(tracking_path)
    pivot_x = df_long.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='x')
    pivot_y = df_long.pivot(index='video_frame', columns=['mouse_id', 'bodypart'], values='y')
    pivot_x.columns = [f"mouse{m}_{bp}_x" for m, bp in pivot_x.columns]
    pivot_y.columns = [f"mouse{m}_{bp}_y" for m, bp in pivot_y.columns]
    df_wide = pd.concat([pivot_x, pivot_y], axis=1).sort_index(axis=1)
    return df_wide


In [ ]:
# --- New Feature Engineering Function ---

# 2. Define a core set of bodyparts to build features from
CORE_BODYPARTS = ['nose', 'ear_left', 'ear_right', 'neck', 'body_center', 'tail_base']

def create_advanced_features(df_wide):
    """
    Creates a rich set of kinematic, interaction, and postural features.
    """
    # Start with a copy of the original data
    features_df = df_wide.copy()
    
    mouse_ids = [1, 2, 3, 4] # Assuming up to 4 mice
    
    # --- 1. Kinematic Features (Speeds) ---
    for mid in mouse_ids:
        for part in CORE_BODYPARTS:
            col_x, col_y = f'mouse{mid}_{part}_x', f'mouse{mid}_{part}_y'
            if col_x in features_df.columns:
                delta_x = features_df[col_x].diff()
                delta_y = features_df[col_y].diff()
                features_df[f'mouse{mid}_{part}_speed'] = np.sqrt(delta_x**2 + delta_y**2)

    # --- 2. Postural Features (Body Elongation) ---
    for mid in mouse_ids:
        nose_x, nose_y = f'mouse{mid}_nose_x', f'mouse{mid}_nose_y'
        tail_x, tail_y = f'mouse{mid}_tail_base_x', f'mouse{mid}_tail_base_y'
        if all(c in features_df.columns for c in [nose_x, nose_y, tail_x, tail_y]):
            features_df[f'mouse{mid}_elongation'] = np.sqrt(
                (features_df[nose_x] - features_df[tail_x])**2 + 
                (features_df[nose_y] - features_df[tail_y])**2
            )

    # --- 3. Interaction Features (Distances) ---
    mouse_pairs = [(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
    for m1, m2 in mouse_pairs:
        for part1 in CORE_BODYPARTS:
            for part2 in CORE_BODYPARTS:
                p1_x, p1_y = f'mouse{m1}_{part1}_x', f'mouse{m1}_{part1}_y'
                p2_x, p2_y = f'mouse{m2}_{part2}_x', f'mouse{m2}_{part2}_y'
                if all(c in features_df.columns for c in [p1_x, p1_y, p2_x, p2_y]):
                    features_df[f'dist_m{m1}{part1}_m{m2}{part2}'] = np.sqrt(
                        (features_df[p1_x] - features_df[p2_x])**2 + 
                        (features_df[p1_y] - features_df[p2_y])**2
                    )
                    
    # Drop the original coordinate columns to force the model to use our new features
    features_df = features_df.drop(columns=df_wide.columns)
    
    return features_df

In [ ]:
# --- Test the function with our sample video ---
print("Testing the feature engineering function...")
df_train_meta = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
sample_video_meta = df_train_meta.iloc[0]

df_wide_sample = load_and_process_video(sample_video_meta['video_id'], sample_video_meta['lab_id'], DATA_PATH)
df_features_sample = create_advanced_features(df_wide_sample)

print("\n--- Function Test Output ---")
if df_features_sample is not None:
    print(f"Successfully created features for video {sample_video_meta['video_id']}")
    print(f"Original coordinate columns: {len(df_wide_sample.columns)}")
    print(f"New engineered feature columns: {len(df_features_sample.columns)}")
    display(df_features_sample.head())
else:
    print("Failed to create features for the sample video.")

## What We Learned in Step 1

*   **Successful Feature Creation:** Our `create_advanced_features` function works as intended. It takes the 142 raw coordinate columns and transforms them into a new set of **225 engineered features**.

*   **Informative Feature Set:** The new columns represent meaningful physical properties:
    *   `mouse1_nose_speed`: How fast a specific bodypart is moving.
    *   `mouse2_elongation`: The posture of a mouse (distance from nose to tail).
    *   `dist_m1nose_m2tail_base`: The spatial relationship between two mice.

*   **Forcing Model to Learn Relationships:** By dropping the original coordinate columns, we are forcing the LightGBM model to learn from these new, relationship-based features. It can no longer learn about absolute positions in the arena, only about speeds, postures, and how the mice are interacting with each other. This is a powerful step towards building a more generalizable model.

**We now have a robust pipeline for converting any video's raw data into a high-quality feature set. The next step is to apply this pipeline to our training data and see if these new features lead to a better model.**

# Step 2: Preparing Data with Advanced Features

**Goal:** To create our full training dataset using the new feature engineering pipeline. The process will be very similar to Notebook 2, but with one key difference: we will call our new `create_advanced_features` function after loading each video.

**Action:**
1.  **Select the Same Subset:** We will use the *exact same* 50 videos as in Notebook 2. This is crucial for a fair comparison, ensuring that any performance difference is due to the features, not the data.
2.  **Load, Process, and Combine:** We will loop through the videos, load the tracking data, create the advanced features for each, and then combine them into a single training DataFrame.
3.  **Create Frame-wise Labels:** We will reuse the exact same logic from Notebook 2 to apply the behavior labels to each frame.

In [ ]:
# --- 1. Select the Same Subset of Videos ---
N_VIDEOS_TO_USE = 50
df_subset_meta = df_train_meta.head(N_VIDEOS_TO_USE)
print(f"Using the same subset of {len(df_subset_meta)} videos for a fair comparison.")

In [ ]:
# --- 2. Load, Process, Create Features, and Combine ---
all_featured_dfs = []
for index, row in tqdm(df_subset_meta.iterrows(), total=len(df_subset_meta)):
    # Load and pivot the data
    df_wide = load_and_process_video(row['video_id'], row['lab_id'], DATA_PATH)
    
    if df_wide is not None:
        # ** NEW STEP: Create advanced features **
        df_features = create_advanced_features(df_wide)
        
        # Add a video_id column for linking annotations
        df_features['video_id'] = row['video_id']
        all_featured_dfs.append(df_features)

# Combine all individual video dataframes into one big one
df_train_full_featured = pd.concat(all_featured_dfs)

print(f"\nLoaded and created features for all videos. Full training shape: {df_train_full_featured.shape}")

In [ ]:
# --- 3. Load Annotations and Create Frame-wise Labels ---
all_annotations_list = []
for video_id in tqdm(df_subset_meta['video_id'].unique(), desc="Loading annotations"):
    row = df_subset_meta[df_subset_meta['video_id'] == video_id].iloc[0]
    annot_path = os.path.join(DATA_PATH, 'train_annotation', row['lab_id'], f"{row['video_id']}.parquet")
    if os.path.exists(annot_path):
        df_annot = pd.read_parquet(annot_path)
        df_annot['video_id'] = video_id
        all_annotations_list.append(df_annot)
df_annotations_subset = pd.concat(all_annotations_list)

# Initialize and apply labels
df_train_full_featured['behavior'] = 'no_behavior'
print("\nApplying annotations to each frame...")
for index, row in tqdm(df_annotations_subset.iterrows(), total=len(df_annotations_subset)):
    video_id, start, stop, action = row['video_id'], row['start_frame'], row['stop_frame'], row['action']
    
    df_train_full_featured.loc[
        (df_train_full_featured['video_id'] == video_id) & 
        (df_train_full_featured.index >= start) & 
        (df_train_full_featured.index <= stop),
        'behavior'
    ] = action

print("Labeling complete.")
print("\nValue counts of the target column (should be identical to Notebook 2):")
print(df_train_full_featured['behavior'].value_counts())

## What We Learned in Step 2

*   **Pipeline Scalability:** Our feature engineering pipeline successfully processed all 50 videos and created a unified training dataset. This demonstrates that our code is robust and can handle the full workflow.

*   **Consistent Labeling:** The `value_counts()` output is identical to Notebook 2. This is a critical sanity check. It confirms that we are training and evaluating on the exact same set of labels, ensuring that any change in model performance will be due to the new features and nothing else.

*   **Ready for Training:** We now have our final training dataset ready. It's a large table where each row is a frame, the columns are our 225 powerful engineered features, and the target is the `behavior` column.

**We are perfectly set up for a direct, apples-to-apples comparison. It's time to train the model and see if our hard work in feature engineering pays off.**

# Step 3: Training the Model on New Features

**Goal:** To train our LightGBM model on the new, feature-rich dataset and compare its performance directly to our baseline.

**Action:**
The code in this step will be almost **identical** to Step 3 in Notebook 2. The only difference is the input DataFrame (`df_train_full_featured`). This intentional similarity allows us to isolate the impact of the features.
1.  Define `X` (our new features) and `y` (the same target).
2.  Handle `NaN` values (especially important for the speed features, which will be `NaN` on the first frame of every video).
3.  Encode labels and split the data, making sure to `stratify` for a fair comparison.
4.  Train the `LGBMClassifier` using the exact same parameters and early stopping. We will be watching the final validation `multi_logloss` very closely.

In [ ]:
# --- 1. Define Features (X) and Target (y) ---
# Our features are all columns EXCEPT 'video_id' and our target 'behavior'
features = [col for col in df_train_full_featured.columns if col not in ['video_id', 'behavior']]
X = df_train_full_featured[features]
y = df_train_full_featured['behavior']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

In [ ]:
# --- 2. Handle Missing Data ---
# The first frame of each video will have NaN for speed features. Fill them with -1.
X = X.fillna(-1)
print("\nFilled NaN values with -1.")

In [ ]:
# --- 3. Encode String Labels into Numbers ---
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print("\nLabels have been encoded.")

In [ ]:
# --- 4. Split Data into Training and Validation Sets ---
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded
)
print(f"\nTraining data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

In [ ]:
# --- 5. Train the LightGBM Model ---
print("\nTraining LightGBM model on NEW features...")

# Use the same model parameters as Notebook 2 for a fair comparison
lgbm_featured = lgb.LGBMClassifier(
    objective='multiclass',
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42,
    n_jobs=-1,
    colsample_bytree=0.8,
    subsample=0.8
)

lgbm_featured.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='multi_logloss',
    callbacks=[lgb.early_stopping(10, verbose=True)]
)

print("\nModel training complete.")

## What We Learned in Step 3

This is a critical and insightful result. Our model with engineered features produced a slightly **worse** validation logloss (0.163) compared to our baseline model with raw coordinates (0.144). This is not a failure; it's a successful experiment that has taught us something profound about our problem.

**Why could this have happened?**

1.  **The 'No Location Information' Hypothesis (Most Likely):** In our feature engineering function, we deliberately dropped the original `x` and `y` coordinate columns. This forced the model to only learn from *relative* information (distances, speeds). However, it's possible that the absolute location of the mice in the arena is a very powerful feature. For example, a mouse in the corner of the cage might be more likely to be `resting` or `selfgrooming` than a mouse in the center. Our baseline model could use that location information, but our new model could not.

2.  **More Features Isn't Always Better:** We added many new features. While LightGBM is good at ignoring useless features, adding too many can sometimes introduce noise or complexity that makes it harder for the model to find the signal, a concept related to the "curse of dimensionality."

**The Critical Question:**
Logloss is a very sensitive metric. It heavily penalizes predictions that are confidently wrong. But is it the whole story? It's possible that while our overall logloss got worse, the model actually got **better** at predicting the rare classes we care about. The `classification_report` will tell us the true story by showing us the per-class F1-scores.

**This result highlights a key lesson: Feature engineering is about finding the *right* features, and sometimes, the simplest ones (like raw position) are unexpectedly powerful.**

# Step 4: In-Depth Evaluation - The True Test

**Goal:** To generate a `classification_report` for our new model. This will allow us to move beyond the single `logloss` number and see the detailed performance (precision, recall, F1-score) for every single behavior. This is where we will find out if our feature engineering was truly a success or not.

**Action:**
1.  Use our new trained model (`lgbm_featured`) to make predictions on the validation set.
2.  Generate the `classification_report`.
3.  **Critically compare** the F1-scores and recall values from this report to the report from Notebook 2. This is the ultimate test of our new features.

In [ ]:
# --- 1. Evaluate Performance on the Validation Set ---
print("--- Model Performance on Validation Set (with Engineered Features) ---")

# Make predictions with the new model
y_pred_featured = lgbm_featured.predict(X_val)

# Convert the numerical predictions back to string labels for the report
y_pred_labels = label_encoder.inverse_transform(y_pred_featured)
y_val_labels = label_encoder.inverse_transform(y_val)

# Generate and print the classification report
report_featured = classification_report(y_val_labels, y_pred_labels)
print(report_featured)

print("\n--- For Comparison: Baseline Report from Notebook 2 ---")
# (I've pasted the key metrics from the previous notebook's output here for easy comparison)
baseline_report_text = """
              precision    recall  f1-score   support

      approach       0.58      0.23      0.32      3568
        attack       0.66      0.18      0.28      7646
         avoid       0.80      0.16      0.27      4256
         chase       0.73      0.22      0.33      3130
   chaseattack       0.69      0.64      0.66      1067
     disengage       0.62      0.28      0.39      2460
dominancemount       0.25      0.52      0.33        91
         mount       0.81      0.62      0.70      2123
   no_behavior       0.97      0.99      0.98   1090760
          rear       0.72      0.46      0.56     15781
     selfgroom       0.79      0.46      0.58      2948
      shepherd       0.69      0.14      0.24      5930
         sniff       0.65      0.54      0.59      7360
     sniffbody       0.35      0.52      0.42       290
     sniffface       0.13      0.18      0.15        79
  sniffgenital       0.71      0.75      0.73       716
        submit       0.76      0.76      0.76      1204

    macro avg       0.64      0.45      0.49   1149409
 weighted avg       0.96      0.96      0.95   1149409
"""
print(baseline_report_text)

## What We Learned in Step 4

This side-by-side comparison is the true result of our experiment. While our overall `macro avg f1-score` went down slightly (from 0.49 to 0.46), the detailed report reveals a fascinating and critical trade-off.

**The Good News: Improved Recall on Key Behaviors**

Our feature-engineered model is "braver" and better at *finding* certain behaviors, even if it's less precise.

*   **`attack`**: Recall jumped from 0.18 -> **0.26**. We are finding significantly more of the true attack events.
*   **`shepherd`**: Recall jumped massively from 0.14 -> **0.24**.
*   **`sniffface` (The Rarest Class!)**: Recall skyrocketed from 0.18 -> **0.63**. The F1-score went from a terrible 0.15 to a respectable **0.40**. This is a huge win! Our new features are clearly helping the model identify this very difficult behavior.
*   **`sniff`**: Recall improved from 0.54 -> **0.66**.

This proves that features describing the *relationships* between mice are essential for identifying interactive behaviors.

**The Bad News: Decreased Precision**

The trade-off for finding more events (higher recall) was a drop in confidence (lower precision) across the board.

*   **`attack`**: Precision dropped from 0.66 -> **0.62**. When the new model says "attack," it's slightly less likely to be correct than the baseline model.
*   **`avoid`**: Precision dropped significantly from 0.80 -> **0.56**.
*   **`selfgroom` & `rear`**: Both recall and precision dropped for these non-interactive behaviors. This supports our hypothesis from Step 3: because these are individual actions, the *absolute position* in the cage (which our new model can't see) might be a very important feature that we mistakenly removed.

**The Grand Conclusion: We Haven't Failed, We've Discovered the Path Forward**

Our experiment was a success. We have proven two things:
1.  **Interaction features (distances, etc.) are crucial for improving RECALL on social behaviors.**
2.  **Raw coordinate features (absolute position) are crucial for improving PRECISION and identifying individual behaviors.**

The obvious next step is to combine the strengths of both models. We need to create a feature set that includes **BOTH** the raw coordinates from Notebook 2 **AND** the engineered features from Notebook 3. This hybrid approach should give us the best of both worlds.

# Step 5: The Hybrid Approach - Combining Feature Sets

**Goal:** Based on our analysis, our final action in this notebook will be to create and train a third model that uses a combined feature set. This represents our best hypothesis so far.

**Action:**
1.  Create a new feature function `create_hybrid_features` that keeps the raw coordinates *and* adds the engineered features.
2.  Quickly retrain a LightGBM model on this new, combined feature set. We don't need to do a full submission pipeline here; we just want to see the validation score from the `classification_report` to confirm our hypothesis.

In [ ]:
# --- 1. Create a Hybrid Feature Function ---
def create_hybrid_features(df_wide):
    """
    Creates a combined set of raw coordinates and engineered features.
    """
    # Start with the original coordinate features from df_wide
    # Create the engineered features, but this time, don't drop the originals
    engineered_features = create_advanced_features(df_wide.copy())
    
    # Combine them
    hybrid_features = pd.concat([df_wide, engineered_features], axis=1)
    return hybrid_features

print("--- Training a Hybrid Model (Best of Both Worlds) ---")

In [ ]:


# --- 2. Build the Hybrid Dataset (using just the first 10 videos for speed) ---
print("\nBuilding a small hybrid dataset for a quick test...")
N_VIDEOS_HYBRID = 10
df_hybrid_meta = df_train_meta.head(N_VIDEOS_HYBRID)

all_hybrid_dfs = []
for index, row in tqdm(df_hybrid_meta.iterrows(), total=len(df_hybrid_meta)):
    df_wide = load_and_process_video(row['video_id'], row['lab_id'], DATA_PATH)
    if df_wide is not None:
        df_hybrid = create_hybrid_features(df_wide)
        df_hybrid['video_id'] = row['video_id']
        all_hybrid_dfs.append(df_hybrid)
df_train_hybrid = pd.concat(all_hybrid_dfs)

# Apply labels
df_train_hybrid['behavior'] = 'no_behavior'
# We only need the annotations for these 10 videos
annot_subset_hybrid = df_annotations_subset[df_annotations_subset['video_id'].isin(df_hybrid_meta['video_id'])]
for index, row in tqdm(annot_subset_hybrid.iterrows(), total=len(annot_subset_hybrid)):
    video_id, start, stop, action = row['video_id'], row['start_frame'], row['stop_frame'], row['action']
    df_train_hybrid.loc[
        (df_train_hybrid['video_id'] == video_id) & (df_train_hybrid.index >= start) & (df_train_hybrid.index <= stop),
        'behavior'
    ] = action

In [ ]:
# --- 3. Train and Evaluate the Hybrid Model ---
features_hybrid = [col for col in df_train_hybrid.columns if col not in ['video_id', 'behavior']]
X_hybrid = df_train_hybrid[features_hybrid].fillna(-1)
y_hybrid = df_train_hybrid['behavior']
y_encoded_hybrid = label_encoder.transform(y_hybrid) # Use the same encoder

X_train_h, X_val_h, y_train_h, y_val_h = train_test_split(
    X_hybrid, y_encoded_hybrid, test_size=0.2, random_state=42, stratify=y_encoded_hybrid
)

print("\nTraining Hybrid LightGBM model...")
lgbm_hybrid = lgb.LGBMClassifier(objective='multiclass', random_state=42, n_jobs=-1) # Using simpler params for speed
lgbm_hybrid.fit(X_train_h, y_train_h)

print("\n--- Hybrid Model Performance ---")
y_pred_hybrid = lgbm_hybrid.predict(X_val_h)
y_pred_labels_h = label_encoder.inverse_transform(y_pred_hybrid)
y_val_labels_h = label_encoder.inverse_transform(y_val_h)
print(classification_report(y_val_labels_h, y_pred_labels_h))

## What We Learned in Step 5 - The Hybrid Triumph

The results from our quick hybrid model experiment are conclusive and overwhelmingly positive.

*   **Massive Performance Leap:** The `macro avg f1-score` jumped to **0.80**. This blows away both the baseline model (0.49) and the engineered-features-only model (0.46). This is a huge improvement.

*   **Best of Both Worlds:** Looking at the detailed scores confirms our theory:
    *   **High Precision:** The precision scores are excellent across the board (mostly 0.89 to 0.94). This indicates that keeping the raw coordinates helped the model be more confident and correct in its predictions.
    *   **High Recall:** The recall scores are also significantly improved compared to our baseline. `approach` is at 0.71 (vs 0.23), `attack` is at 0.60 (vs 0.18), and `rear` is at 0.85 (vs 0.46). This proves the engineered features are helping the model *find* the behaviors it was previously missing.

*   **A Clear Path Forward:** We have found a winning formula for our feature set. The combination of absolute positional information (raw coordinates) and relative interaction information (engineered features) is far more powerful than either one in isolation.

# Notebook 3 Conclusion: The Power of Informed Features

This notebook was a fantastic demonstration of the iterative nature of data science. We didn't just build one model; we conducted a series of experiments that taught us valuable lessons.

### Key Accomplishments:
1.  **Isolated the Impact of Features:** By comparing our baseline to a feature-only model, we learned the specific strengths and weaknesses of different feature types.
2.  **Discovered a Winning Combination:** Our final hybrid model, combining raw coordinates and engineered features, proved to be vastly superior, achieving a **Macro F1-score of 0.80** in our test.
3.  **Created a Powerful Feature Set:** We now have a robust feature engineering pipeline that can serve as the foundation for all future, more advanced models.

This feature set is so powerful that it will likely be the one we carry forward into the next notebooks. We have successfully climbed a significant step up the performance ladder.

**Next Up: Notebook 4 - Thinking in Time: Introduction to Sequence Models (LSTM/GRU)**
We have pushed the frame-by-frame approach about as far as it can go. In the next notebook, we will tackle the final fundamental weakness of our approach: ignoring the dimension of time. We will take our new hybrid feature set and feed it into a model that can understand sequences.